In [9]:
import numpy as np
import pandas as pd

import cfbd

import requests
import json
import os
import sys

from datetime import datetime as dt

In [12]:
# Local Imports
from resources.helper import load_configs, authenticate_api
from preprocessing.data_gathering import get_game_stats, get_games, get_betting_info
from preprocessing.preprocessing import get_simple_rolling_stats, create_training_set, get_latest_feature_values

In [13]:
# Load configs
api_configs_path = "/Users/taylorlast/PersonalProjects/cfb-predictions/configs/api_configs.yml"
api_configs = load_configs(config_path=api_configs_path)

# Auth API
api_configuration = authenticate_api(api_key=api_configs["API_KEY"])

In [18]:
games = get_games(configuration=api_configuration, year=2023, only_fbs=True, week=1)

In [19]:
games

,id,season,week,home_team,away_team,home_points,away_points,home_division,away_division,home_pregame_elo,away_pregame_elo,neutral_site,point_diff,winner
0,401525434,2023,1,Notre Dame,Navy,42.0,3.0,fbs,fbs,1826.0,1403.0,True,39.0,Notre Dame
1,401520145,2023,1,Jacksonville State,UTEP,17.0,14.0,fbs,fbs,NaN,NaN,False,3.0,Jacksonville State
2,401525450,2023,1,New Mexico State,UMass,30.0,41.0,fbs,fbs,1184.0,1036.0,False,-11.0,UMass
3,401532392,2023,1,San Diego State,Ohio,20.0,13.0,fbs,fbs,1535.0,1368.0,False,7.0,San Diego State
4,401520147,2023,1,Vanderbilt,Hawai'i,35.0,28.0,fbs,fbs,1222.0,1432.0,False,7.0,Vanderbilt
5,401523986,2023,1,USC,San José State,56.0,28.0,fbs,fbs,1487.0,1338.0,False,28.0,USC
6,401520146,2023,1,Louisiana Tech,Florida International,22.0,17.0,fbs,fbs,1301.0,1080.0,False,5.0,Louisiana Tech
7,401525816,2023,1,UCF,Kent State,56.0,6.0,fbs,fbs,1569.0,1340.0,False,50.0,UCF
8,401520174,2023,1,Connecticut,NC State,14.0,24.0,fbs,fbs,1117.0,1632.0,False,-10.0,NC State
9,401520148,2023,1,Minnesota,Nebraska,13.0,10.0,fbs,fbs,1639.0,1636.0,False,3.0,Minnesota


In [20]:
stats = get_game_stats(configuration=api_configuration, year=2023, week=1)

In [27]:
lines = get_betting_info(configuration=api_configuration, year=2023, week=1)

In [28]:
lines

,id,season,week,season_type,start_date,home_team,home_conference,home_score,away_team,away_conference,away_score,lines,consensus_spread(reversed)
0,401540508,2023,1,regular,2023-09-02T17:00:00.000Z,Stonehill,NEC,17.0,New Hampshire,CAA,51.0,"[{'provider': 'Bovada', 'spread': 21.0, 'forma...",-21.0
1,401540200,2023,1,regular,2023-09-02T23:00:00.000Z,North Alabama,UAC,41.0,Chattanooga,Southern,27.0,"[{'provider': 'Bovada', 'spread': 19.5, 'forma...",-19.5
2,401540148,2023,1,regular,2023-09-03T19:00:00.000Z,Florida A&M,SWAC,28.0,Jackson State,SWAC,10.0,"[{'provider': 'Bovada', 'spread': 3.5, 'format...",-3.5
3,401540099,2023,1,regular,2023-09-02T17:00:00.000Z,Illinois State,MVFC,41.0,Dayton,Pioneer,0.0,"[{'provider': 'Bovada', 'spread': -19.0, 'form...",19.0
4,401540057,2023,1,regular,2023-09-02T16:30:00.000Z,Georgetown,Patriot,49.0,Marist,Pioneer,7.0,"[{'provider': 'Bovada', 'spread': -7.0, 'forma...",7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,401532392,2023,1,regular,2023-08-26T23:00:00.000Z,San Diego State,Mountain West,20.0,Ohio,Mid-American,13.0,"[{'provider': 'Bovada', 'spread': -2.5, 'forma...",2.5
99,401525450,2023,1,regular,2023-08-26T23:00:00.000Z,New Mexico State,Conference USA,30.0,UMass,FBS Independents,41.0,"[{'provider': 'Bovada', 'spread': -7.5, 'forma...",7.5
100,401525434,2023,1,regular,2023-08-26T18:30:00.000Z,Notre Dame,FBS Independents,42.0,Navy,American Athletic,3.0,"[{'provider': 'Bovada', 'spread': -20.5, 'form...",20.5
101,401523986,2023,1,regular,2023-08-27T00:00:00.000Z,USC,Pac-12,56.0,San José State,Mountain West,28.0,"[{'provider': 'Bovada', 'spread': -30.5, 'form...",30.5


In [9]:
stats

,id,season,week,team,opponent,plays_offense,drives_offense,ppa_offense,total_ppa_offense,success_rate_offense,...,passing_downs.success_rate_defense,passing_downs.explosiveness_defense,rushing_plays.ppa_defense,rushing_plays.total_ppa_defense,rushing_plays.success_rate_defense,rushing_plays.explosiveness_defense,passing_plays.ppa_defense,passing_plays.total_ppa_defense,passing_plays.success_rate_defense,passing_plays.explosiveness_defense
0,401403853,2022,1,Hawai'i,Vanderbilt,80,15,-0.196504,-15.720358,0.375000,...,0.380952,2.284470,0.615889,26.483238,0.604651,1.397073,0.047978,1.247420,0.384615,1.771888
1,401403853,2022,1,Vanderbilt,Hawai'i,69,13,0.401894,27.730658,0.521739,...,0.111111,1.539600,-0.363221,-8.717298,0.458333,1.142734,-0.125055,-7.003060,0.339286,1.043512
2,401403854,2022,1,Alabama,Utah State,72,13,0.461649,33.238746,0.541667,...,0.181818,1.597720,-0.202975,-7.510077,0.243243,1.139472,-0.282363,-6.211983,0.181818,1.197334
3,401403854,2022,1,Utah State,Alabama,59,13,-0.232577,-13.722060,0.220339,...,0.529412,2.377240,0.348267,10.796292,0.580645,1.144513,0.547377,22.442454,0.512195,1.653114
4,401403855,2022,1,Arkansas,Cincinnati,70,12,0.229694,16.078613,0.385714,...,0.320000,2.268853,0.155747,4.360914,0.428571,0.983519,0.296841,13.357827,0.400000,1.900264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2837,401508851,2022,1,South Dakota State,Montana State,50,8,0.598341,29.917065,0.640000,...,0.277778,2.973462,-0.100625,-2.817506,0.357143,0.828034,0.435416,10.885391,0.480000,1.790262
2838,401508870,2022,1,Colorado Mines,Ferris State,53,13,-0.269881,-14.303693,0.339623,...,0.411765,1.560169,0.266709,10.935056,0.512195,0.968332,0.380688,7.233071,0.473684,1.572327
2839,401508870,2022,1,Ferris State,Colorado Mines,60,11,0.302802,18.168127,0.500000,...,0.105263,1.460110,-0.463078,-6.483088,0.285714,0.758980,-0.200528,-7.820605,0.358974,1.116276
2840,401508871,2022,1,Mount Union,North Central,72,10,0.132431,9.535044,0.416667,...,0.181818,2.539994,0.120254,4.689899,0.307692,1.679884,1.026566,10.265664,0.400000,3.459609


In [10]:
games

,id,season,week,home_team,away_team,home_points,away_points,home_division,away_division,home_pregame_elo,away_pregame_elo,neutral_site,point_diff,winner
0,401405059,2022,1,Northwestern,Nebraska,31.0,28.0,fbs,fbs,1361.0,1636.0,True,3.0,Northwestern
1,401405058,2022,1,Illinois,Wyoming,38.0,6.0,fbs,fbs,1496.0,1497.0,False,32.0,Illinois
2,401404146,2022,1,Utah State,Connecticut,31.0,20.0,fbs,fbs,1566.0,1117.0,False,11.0,Utah State
3,401426530,2022,1,Florida Atlantic,Charlotte,43.0,13.0,fbs,fbs,1427.0,1272.0,False,30.0,Florida Atlantic
4,401426531,2022,1,UTEP,North Texas,13.0,31.0,fbs,fbs,1360.0,1426.0,False,-18.0,North Texas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,401437034,2022,14,Boise State,Fresno State,16.0,28.0,fbs,fbs,1750.0,1676.0,False,-12.0,Fresno State
734,401437036,2022,14,Georgia,LSU,50.0,30.0,fbs,fbs,2132.0,1729.0,True,20.0,Georgia
735,401437031,2022,14,Michigan,Purdue,43.0,22.0,fbs,fbs,2144.0,1622.0,True,21.0,Michigan
736,401437009,2022,14,North Carolina,Clemson,10.0,39.0,fbs,fbs,1599.0,1821.0,True,-29.0,Clemson


In [11]:
lines

,id,season,week,season_type,start_date,home_team,home_conference,home_score,away_team,away_conference,away_score,lines,consensus_spread(reversed)
0,401411101,2022,1,regular,2022-09-06T00:00:00.000Z,Georgia Tech,ACC,10.0,Clemson,ACC,41.0,"[{'provider': 'consensus', 'spread': 24.5, 'fo...",-24.5
1,401404079,2022,4,regular,2022-09-24T16:00:00.000Z,Kansas,Big 12,35.0,Duke,ACC,27.0,"[{'provider': 'consensus', 'spread': -7.0, 'fo...",7.0
2,401403893,2022,4,regular,2022-09-24T23:00:00.000Z,Texas A&M,SEC,23.0,Arkansas,SEC,21.0,"[{'provider': 'consensus', 'spread': -1.5, 'fo...",1.5
3,401404080,2022,4,regular,2022-09-24T16:00:00.000Z,Iowa State,Big 12,24.0,Baylor,Big 12,31.0,"[{'provider': 'consensus', 'spread': -2.5, 'fo...",2.5
4,401404081,2022,4,regular,2022-09-25T00:00:00.000Z,Oklahoma,Big 12,34.0,Kansas State,Big 12,41.0,"[{'provider': 'consensus', 'spread': -13.5, 'f...",13.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,401404090,2022,6,regular,2022-10-08T16:00:00.000Z,Oklahoma,Big 12,0.0,Texas,Big 12,49.0,"[{'provider': 'consensus', 'spread': 7.5, 'for...",-7.5
1197,401404011,2022,6,regular,2022-10-08T19:30:00.000Z,UCLA,Pac-12,42.0,Utah,Pac-12,32.0,"[{'provider': 'consensus', 'spread': 3.0, 'for...",-3.0
1198,401404012,2022,6,regular,2022-10-08T23:30:00.000Z,USC,Pac-12,30.0,Washington State,Pac-12,14.0,"[{'provider': 'consensus', 'spread': -12.5, 'f...",12.5
1199,401404088,2022,6,regular,2022-10-08T23:30:00.000Z,Iowa State,Big 12,9.0,Kansas State,Big 12,10.0,"[{'provider': 'consensus', 'spread': 1.0, 'for...",-1.0


In [12]:
df = pd.merge(games, lines[["id", "home_team", "consensus_spread(reversed)"]], on=["id", "home_team"])

In [13]:
df["diff"] = df["point_diff"] - df["consensus_spread(reversed)"]

# If diff is positive, the home team won by more than they should've based on vegas spread.

In [14]:
model_cols = [
    "id",
    "home_team",
    "away_team",
    "home_pregame_elo",
    "away_pregame_elo",
    "neutral_site",
    "consensus_spread(reversed)",
    "diff",
]
df[(df["home_team"]=="Georgia") | (df["away_team"]=="Georgia")][model_cols]

,id,home_team,away_team,home_pregame_elo,away_pregame_elo,neutral_site,consensus_spread(reversed),diff
24,401403858,Georgia,Oregon,1980.0,1574.0,True,16.0,30.0
102,401403888,South Carolina,Georgia,1521.0,2070.0,False,-25.5,-15.5
154,401403895,Georgia,Kent State,2121.0,1274.0,False,45.0,-28.0
257,401403907,Missouri,Georgia,1411.0,2090.0,False,-30.5,26.5
287,401403911,Georgia,Auburn,2058.0,1558.0,False,28.0,4.0
345,401403918,Georgia,Vanderbilt,2087.0,1338.0,False,37.5,17.5
444,401403928,Georgia,Florida,2154.0,1553.0,True,23.0,-1.0
492,401403933,Georgia,Tennessee,2155.0,1970.0,False,10.0,4.0
572,401403944,Mississippi State,Georgia,1688.0,2168.0,False,-16.5,-9.5
620,401403949,Kentucky,Georgia,1583.0,2184.0,False,-22.5,12.5


In [ ]:
# TODO: Previous seasons don't change so store that in a data frame and load it in instead of calling everytime.
# TODO: Load week by week and keep an up-to-date feature store so inference is quick.
# TODO: Keep track of current week in a txt file and write a function to update current week
# TODO: Join features and spreads on games
# TODO: Fit a model using features and store the model
# TODO: Write in a function that reads in the model every time the inference job is run.
# TODO: Write a job that retrains every 'x' weeks.

In [9]:
api_instance = cfbd.StatsApi(cfbd.ApiClient(api_configuration))

stats = api_instance.get_advanced_team_game_stats(year=2023)

In [5]:
test_df = pd.read_csv("/Users/taylorlast/PersonalProjects/cfb-predictions/data/stats_df.csv")
games_df = pd.read_csv("/Users/taylorlast/PersonalProjects/cfb-predictions/data/games_df.csv")

In [6]:
get_latest_feature_values(test_df)

/Users/taylorlast/PersonalProjects/cfb-predictions/src/preprocessing/preprocessing.py:114: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['opponent'], dtype='object')
  stats_df


,team,id,plays_offense,drives_offense,ppa_offense,total_ppa_offense,success_rate_offense,explosiveness_offense,power_success_offense,stuff_rate_offense,...,passing_downs.success_rate_defense,passing_downs.explosiveness_defense,rushing_plays.ppa_defense,rushing_plays.total_ppa_defense,rushing_plays.success_rate_defense,rushing_plays.explosiveness_defense,passing_plays.ppa_defense,passing_plays.total_ppa_defense,passing_plays.success_rate_defense,passing_plays.explosiveness_defense
0,Abilene Christian,401462519,66.142857,12.571429,0.213261,13.497205,0.405540,1.394034,0.621429,0.303604,...,0.324875,2.240784,0.046685,1.786987,0.407978,1.192869,0.317828,11.654470,0.412646,1.614204
1,Air Force,401442040,64.714286,10.000000,0.236201,14.860314,0.455494,1.094662,0.855442,0.089168,...,0.227882,2.252376,0.104683,2.111586,0.391019,0.938435,0.275236,5.032431,0.440902,1.425303
2,Akron,401416652,67.142857,11.857143,0.121093,9.102121,0.440189,1.238823,0.795918,0.160948,...,0.322400,2.177122,0.083174,3.658303,0.427755,0.942706,0.424230,10.658207,0.487214,1.564592
3,Alabama,401442018,69.857143,11.857143,0.305620,21.052245,0.461021,1.359253,0.763946,0.206202,...,0.275606,1.946826,0.192473,6.689322,0.450414,0.976840,0.042169,0.925159,0.370782,1.365572
4,Alabama A&M,401426534,64.571429,11.714286,0.102879,7.316950,0.422710,1.166801,0.789116,0.175212,...,0.344760,2.067794,0.262269,7.610160,0.481103,1.174610,0.167808,6.905666,0.399145,1.634346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,Wisconsin,401442005,63.142857,12.285714,0.130974,7.459375,0.404365,1.265105,0.623129,0.172435,...,0.284143,2.019101,-0.048113,-1.517300,0.318773,0.952957,0.217476,7.109616,0.375685,1.587073
294,Wofford,401413212,54.714286,11.571429,0.258015,11.003969,0.402783,1.512292,0.597506,0.209472,...,0.321882,2.498821,0.303503,8.648024,0.475688,1.136677,0.682233,12.076789,0.517516,2.073094
295,Wyoming,401443543,59.142857,12.285714,0.126860,8.066897,0.400036,1.248483,0.685714,0.178343,...,0.273129,2.064321,0.216196,7.439896,0.472566,0.936315,0.052481,1.891226,0.326374,1.573675
296,Yale,401431402,67.428571,12.285714,0.276471,18.960354,0.458054,1.294414,0.911565,0.103369,...,0.253529,1.991057,0.080648,1.786067,0.406086,1.155106,0.130003,6.290752,0.370169,1.435030


In [13]:
test_df = (
        test_df
        .set_index("id")
        .groupby("team")
        .rolling(7, min_periods=1)
        .mean()
        .groupby("team")
        .shift(1)
        .reset_index()
        .drop(["season", "week", "Unnamed: 0"], axis=1)
)

/var/folders/6y/6p7qrh9s27d1s3l8p2l81vbw0000gn/T/ipykernel_50044/3254997284.py:2: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['opponent'], dtype='object')
  test_df


In [15]:
test_df[test_df["id"] == 400945247]

,team,id,plays_offense,drives_offense,ppa_offense,total_ppa_offense,success_rate_offense,explosiveness_offense,power_success_offense,stuff_rate_offense,...,passing_downs.success_rate_defense,passing_downs.explosiveness_defense,rushing_plays.ppa_defense,rushing_plays.total_ppa_defense,rushing_plays.success_rate_defense,rushing_plays.explosiveness_defense,passing_plays.ppa_defense,passing_plays.total_ppa_defense,passing_plays.success_rate_defense,passing_plays.explosiveness_defense
2,Abilene Christian,400945247,68.500000,14.000000,0.084416,5.096345,0.312956,1.507254,0.250000,0.224856,...,0.376471,1.953956,0.282494,14.709521,0.510691,1.024549,0.564578,8.379752,0.427778,2.186032
7681,New Mexico,400945247,65.571429,11.571429,0.235153,13.794917,0.440560,1.235583,0.784524,0.191140,...,0.408912,1.895525,0.199363,7.971423,0.430624,1.123343,0.480970,12.749474,0.501088,1.606548


In [9]:
# Load configs
api_configs_path = "/Users/taylorlast/PersonalProjects/cfb-predictions/configs/api_configs.yml"
api_configs = load_configs(config_path=api_configs_path)

# Auth API
api_configuration = authenticate_api(api_key=api_configs["API_KEY"])

In [17]:
api_instance =cfbd.BettingApi(cfbd.ApiClient(api_configuration))
spreads = api_instance.get_lines(year=2023)

In [20]:
spreads = api_instance.get_lines(year=2023, week=1)

In [23]:
df = pd.DataFrame().from_records([s.to_dict() for s in spreads])

In [27]:
df[df["home_team"] == "Notre Dame"]["lines"][331]

[{'provider': 'DraftKings',
  'spread': -20.0,
  'formatted_spread': 'Notre Dame -20',
  'spread_open': -20.0,
  'over_under': 50.5,
  'over_under_open': 50.5,
  'home_moneyline': -1350.0,
  'away_moneyline': 800.0},
 {'provider': 'Bovada',
  'spread': -20.5,
  'formatted_spread': 'Notre Dame -20.5',
  'spread_open': -19.5,
  'over_under': 50.5,
  'over_under_open': 49.0,
  'home_moneyline': -1500.0,
  'away_moneyline': 775.0}]

In [39]:
df = pd.read_csv("/Users/taylorlast/PersonalProjects/cfb-predictions/data/current_predictions.csv")

In [40]:
df["diff_in_spread"] = np.abs(df["predicted_diff"])
df.sort_values(by="diff_in_spread", ascending=False)[["home_team", "away_team", "consensus_spread(reversed)", "spread_prediction", "spread"]]

,home_team,away_team,consensus_spread(reversed),spread_prediction,spread
20,South Florida,Alabama,-34.0,16.048426,34.0
12,Old Dominion,Wake Forest,-14.0,30.656406,14.0
24,Iowa,Western Michigan,28.0,-11.360338,-28.0
2,UT San Antonio,Army,7.0,-23.118406,-7.0
34,Oklahoma State,South Alabama,7.0,3.448458,-7.0
50,Colorado,Colorado State,24.0,-13.752935,-24.0
3,Air Force,Utah State,8.5,-18.703294,-8.5
49,Houston,TCU,-7.5,-2.290425,7.5
36,Toledo,San José State,7.5,1.885179,-7.5
31,Cincinnati,Miami (OH),14.0,-22.280478,-14.0


In [41]:
df = pd.read_csv("/Users/taylorlast/PersonalProjects/cfb-predictions/data/current_predictions.csv")[["id", "home_team", "away_team", "spread_prediction", "spread"]]

In [42]:
df["delta"] = np.abs(df["spread_prediction"] - df["spread"])

In [43]:
df.sort_values(by="delta", ascending=False).to_csv("juttin.csv")